In [ ]:
## Calculate and plot the stresses 

## Create an array with zeros, size of n_elem
sigma_trivec = np.zeros((n_elem,3))
for i in range(n_elem):
     # Get the nodes making up this element
    node1 = elements[i,0]-1 # the -1 is because indices start from 0
    node2 = elements[i,1]-1 # the -1 is because indices start from 0
    node3 = elements[i,2]-1 # the -1 is because indices start from 0

    # Get the coordinates of the triangle
    x1 = node_X[node1,0]
    y1 = node_X[node1,1]
    x2 = node_X[node2,0]
    y2 = node_X[node2,1]
    x3 = node_X[node3,0]
    y3 = node_X[node3,1]
    Ae = Area(x1,y1,x2,y2,x3,y3)
    
    # B matrix
    B = (1./(2.*Ae))*np.array([[y2-y3,0,y3-y1,0,y1-y2,0],
                               [0,x3-x2,0,x1-x3,0,x2-x1],
                               [x3-x2,y2-y3,x1-x3,y3-y1,x2-x1,y1-y2]])
    
    # D matrix
    D = E/(1-nu**2)*np.array([[1.,nu,0.],[nu,1.,0.],[0.,0.,(1.-nu)/2.]])
    
    ## KEY PART OF THIS CODE ## 
    ## to evaluate the stress need to go get the solution of the displacement
    ## That is, if you have your solution stored in the vector 'd' which should be size 2*n_node x 1
    ## this vector has the displacements of the nodes, here you need to 'retreive' the displacement of
    ## the nodes that form this element, it is sort of the opposite you do in the assembly
    
    node_e = [node1,node2,node3]
    ue = np.zeros((6))
    for ni in range(3):
        ue[ni*2+0] = d[node_e[ni]*2+0]
        ue[ni*2+1] = d[node_e[ni]*2+1]
            
    ## so now you have the displacements of the nodes making up this element, and they are arranged
    ## in a 6x1 vector, the strain is actually strain = epsilon = B^e*u^e
    ## note that because B^e is 3x6 and u^e is 6x1, epsilon is 3x1, actually, if you check in the book this is
    ## epsilon = [epsilon_xx, epislon_yy, epsilon_xy]
    epsilon = np.dot(B,ue)
    
    ## the stress is nothing more than sigma = D*epsilon, D is a 3x3 matrix, so sigma is 3x1
    ## the entries of this vector are sigma = [sigma_xx, sigma_yy, sigma_xy]
    sigma = np.dot(D,epsilon)
    
    ## Here we just store the sigma of this element to the big vector of size n_elem x 3
    ## which will have the stress of all elements
    sigma_trivec[i,:] = sigma